# vorbereitung

In [1]:
import numpy as np
import trimesh as tm
import pandas
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import networkx as nx
import sys
from sklearn.cluster import SpectralClustering
from sklearn import metrics
from scripts import Segmentation_helper as sh
import scipy.sparse
import scipy.linalg
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400)

In [2]:
dumbell = tm.load("../Meshes/dumbell.stl", force='mesh')
box = tm.load("../Meshes/box.ply", force='mesh')
airplane = tm.load("../Meshes/MeshsegBenchmark-1.0/data/off/61.off", force='mesh')
bearing = tm.load("../Meshes/MeshsegBenchmark-1.0/data/off/341.off", force='mesh')
bunny = tm.load("../Meshes/bunny10k_solid.obj", force='mesh')
plane_part_1 = tm.load("../Meshes/plane_part_1.stl", force='mesh')

In [3]:
mesh = box
n_segments = 2
save_submeshes = True
file_location = '../submeshes'

# Mesh bearbeitung
## coarse segmentation

### angular distance

In [4]:
convex_list= mesh.face_adjacency_convex.copy().tolist()

In [5]:
face_angle = mesh.face_adjacency_angles.copy()

In [6]:
angular_distance_1 = (1+np.cos(face_angle.round(3).copy())).round(3)

In [7]:
mu = 0.1
for i in range(len(convex_list)):
    if convex_list[i]:
        angular_distance_1[i] *= mu

In [8]:
norm_angular_distance_1 = angular_distance_1/angular_distance_1.mean()

In [9]:
vector = np.subtract(*mesh.vertices[mesh.face_adjacency_edges.T])

In [10]:
length = tm.util.row_norm(vector)

In [11]:
similarity_factor = np.exp(-1*norm_angular_distance_1)

In [12]:
similarity_factor = (similarity_factor*length).round(3)

### adjacenz matrix

In [13]:
graph = nx.from_edgelist(mesh.face_adjacency)

In [14]:
edge_weight = dict(zip([tuple(x) for x in mesh.face_adjacency],similarity_factor))

In [15]:
nx.set_edge_attributes(graph, values = edge_weight, name = 'weight')

In [16]:
adj_matrix = nx.to_numpy_matrix(graph, nodelist=sorted(graph.nodes()))

### laplace Matrix

In [17]:
# laplacian mit scipy ausrechenen
adj_matrix_sparce = nx.to_scipy_sparse_matrix(graph, nodelist=sorted(graph.nodes()))
test = scipy.sparse.csgraph.laplacian(adj_matrix_sparce, normed=False)
test.todense()

matrix([[ 2.521,  0.   , -0.945,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   , -0.945,  0.   ],
        [-0.945,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.631, -0.945, -0.945],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   , -0.945,  0.   ,  0.   ,  0.   , -0.631, -0.945],
        [-0.945,  0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   ,  0.   , -0.945,  0.   ,  0.   , -0.631],
        [-0.631, -0.945,  0.   ,  0.   , -0.945,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   , -0.631, -0.945,  0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   , -0.945,  0.   ,  0.   ],
        [ 0.   ,  0.   , -0.631,  0.   ,  0.   , -0.945,  0.   ,  0.   ,  2.521, -0.945,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   , -0.631,  0.  

In [18]:
# lap mit networkx ausrechnen, beides aequivalent
lap = nx.linalg.laplacianmatrix.laplacian_matrix(G = graph, nodelist=sorted(graph.nodes()))
lap.todense()

matrix([[ 2.521,  0.   , -0.945,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   , -0.945,  0.   ],
        [-0.945,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   , -0.945, -0.631,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.631, -0.945, -0.945],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   , -0.945,  0.   ,  0.   ,  0.   , -0.631, -0.945],
        [-0.945,  0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   ,  0.   , -0.945,  0.   ,  0.   , -0.631],
        [-0.631, -0.945,  0.   ,  0.   , -0.945,  0.   ,  2.521,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   , -0.631, -0.945,  0.   ,  0.   ,  0.   ,  0.   ,  2.521,  0.   , -0.945,  0.   ,  0.   ],
        [ 0.   ,  0.   , -0.631,  0.   ,  0.   , -0.945,  0.   ,  0.   ,  2.521, -0.945,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   , -0.631,  0.  

### u_snake ausrechen

In [19]:
vals, vecs = scipy.sparse.linalg.eigsh(lap, k = 2, which = 'SM')

In [20]:
vals = vals.round(3)
vecs = vecs.round(3)

In [21]:
fiedler_vector = vecs[:,1]
fiedler_vector

array([-0.235,  0.235, -0.374,  0.235,  0.374, -0.235,  0.235, -0.235, -0.374, -0.235,  0.374,  0.235])

In [22]:
phi = np.argsort(fiedler_vector)
phi_inv = np.argsort(phi)

In [23]:
sorted_fiedler_vector = fiedler_vector[phi]
first_diff = np.diff(fiedler_vector[phi])

In [28]:
jump_index_tmp = np.argsort(-(first_diff))
jump_index = jump_index_tmp[:n_segments-1]
jump_index = jump_index[np.argsort(jump_index)]
jump_index = jump_index +1 
jump_index = np.insert(jump_index,0,0)
jump_index = np.append(jump_index,len(mesh.faces))

c = np.zeros(n_segments)
for i in range(n_segments):
    ctmp = np.sum(sorted_fiedler_vector[jump_index[i]:jump_index[i+1]])
    c[i] = ctmp/(jump_index[i+1]-jump_index[i])

u_snake_sorted = np.repeat(c,np.diff(jump_index))
u_snake = u_snake_sorted[phi_inv]

In [30]:
u_snake_sorted.round(3)

array([-0.281, -0.281, -0.281, -0.281, -0.281, -0.281,  0.281,  0.281,  0.281,  0.281,  0.281,  0.281])

In [31]:
sorted_fiedler_vector

array([-0.374, -0.374, -0.235, -0.235, -0.235, -0.235,  0.235,  0.235,  0.235,  0.235,  0.374,  0.374])